#Image segmentation - Feulgen Stained Nuclei

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!/opt/bin/nvidia-smi
!nvcc --version

In [0]:
from fastai.vision import *
from fastai.callbacks import SaveModelCallback
from pathlib import Path
torch.backends.cudnn.benchmark=True
from fastai.callbacks import SaveModelCallback

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
path = Path('gdrive/My Drive/Colab Notebooks/citologia_colo_de_utero/dataset')
path.ls()

In [0]:
path_lbl = path/'labels'
path_img = path/'images'

## Data

In [0]:
fnames = get_image_files(path_img/'train')
fnames[:3]
len(fnames)

In [0]:
lbl_names = get_image_files(path_lbl)
lbl_names[:3]
len(lbl_names)

In [0]:
codes = np.loadtxt(path/'codes.txt', dtype=str); codes

In [0]:
img_f = fnames[2]
img = open_image(img_f)
img.show(figsize=(5,5))
img_f

In [0]:
get_y_fn = lambda x: path_lbl/f'{x.stem}{x.suffix}'

In [0]:
mask = open_mask(get_y_fn(img_f))
mask.show(figsize=(5,5), alpha=1)

In [0]:
src_size = np.array(mask.shape[1:])
src_size, mask

## IOU metric, Initial data split and model 

In [0]:
name2id = {v:k for k,v in enumerate(codes)}

def iou_metric(input, target):
    target = target.squeeze(1)
    mask = target != 0
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()

In [0]:
src = (SegmentationItemList.from_folder(path_img)
       .split_by_folder(train='train', valid='valid') 
       .label_from_func(get_y_fn, classes=codes))

In [0]:
wd=1e-3
metrics = iou_metric
used_model = models.resnet34

## Training...

###300 x 400

In [0]:
size = src_size//4
print(size)
bs=16

In [0]:
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(imagenet_stats))

In [0]:
learn = unet_learner(data, used_model, metrics=metrics, wd=wd)

In [0]:
lr_find(learn)
learn.recorder.plot()

In [0]:
lr=1e-2

If you're resuming, only indicating the epoch from which to resume, indicated by start_epoch=<epoch#> will load the last saved .pth, it is not necessary to explicitly reload the last epoch, you only should NOT change the name given in 
name=<callback_save_file>: when resuming fast.ai will try to reload <callback_save_file>_<previous_epoch>.pth

Use start_epoch=<some_epoch> to resume training...

**start_epoch = last_checkpoint_saved + 1**

In [0]:
learn.fit_one_cycle(5, max_lr=slice(lr), 
                    start_epoch=0,
                    callbacks=[SaveModelCallback(learn, 
                    every='epoch', monitor='accuracy', 
                    name='stage1-300x400')])

In [0]:
learn.save('stage1-300x400')

####Unfreeze

In [0]:
learn.load('stage1-300x400')

In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(10, max_lr=slice(lr/10, lr), 
                    start_epoch=0,
                    callbacks=[SaveModelCallback(learn, 
                    every='epoch', monitor='accuracy', 
                    name='stage2-300x400')])

In [0]:
learn.save('stage2-300x400')

### 600 x 800

In [0]:
size = src_size//2
bs=4

In [0]:
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(imagenet_stats))

In [0]:
learn = unet_learner(data, used_model, metrics=metrics, wd=wd)

In [0]:
learn.load('stage2-300x400')

In [0]:
lr_find(learn)
learn.recorder.plot()

In [0]:
lr=1e-4

In [0]:
learn.fit_one_cycle(5, max_lr=slice(lr), 
                    start_epoch=0,
                    callbacks=[SaveModelCallback(learn, 
                    every='epoch', monitor='accuracy', 
                    name='stage1-600x800')])

In [0]:
learn.save('stage1-600x800')

#### Unfreeze


In [0]:
learn.load('stage1-600x800')

In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(10, max_lr=slice(lr/10, lr), 
                    start_epoch=0,
                    callbacks=[SaveModelCallback(learn, 
                    every='epoch', monitor='accuracy', 
                    name='stage2-600x800')])

In [0]:
learn.save('stage2-600x800')

### 1200 x 1600

In [0]:
size = src_size
bs=1

In [0]:
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(imagenet_stats))

In [0]:
learn = unet_learner(data, used_model, metrics=metrics, wd=wd)

In [0]:
learn.load('stage2-600x800')

In [0]:
lr_find(learn)
learn.recorder.plot()

In [0]:
lr=1e-5

In [0]:
learn.fit_one_cycle(5, max_lr=slice(lr), 
                    start_epoch=0,
                    callbacks=[SaveModelCallback(learn, 
                    every='epoch', monitor='accuracy', 
                    name='stage1-1200x1600')])

In [0]:
learn.save('stage1-1200x1600')

#### Unfreeze


In [0]:
learn.load('stage1-1200x1600')

In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(10, max_lr=slice(lr/10, lr), 
                    start_epoch=0,
                    callbacks=[SaveModelCallback(learn, 
                    every='epoch', monitor='accuracy', 
                    name='stage2-1200x1600')])

In [0]:
learn.save('stage2-1200x1600')